In [1]:
## Loading libraries

from turtle import width
import torch
import torch.distributions as td
from torch.autograd import Variable

import plotly.express as px

import numpy as np
from tqdm.notebook import trange
torch.set_default_tensor_type(torch.DoubleTensor)

margin_dic = dict(l=10, r= 10, t = 30, b = 50)
w,h = 600, 300
from math import pi
import os

## Simple Bayesian Logistic

\begin{equation}
\begin{gathered}
\beta \sim N\left(0, \sigma^2 I_p\right), ~ p = 4\\
\mathbf{P}\left(y_i = 1 | x_i, \beta \right) = 1 - \mathbf{P}\left(y_i = 0 | x_i, \beta \right) = \sigma\left(x_i^T \beta\right) = \frac{1}{1 + \exp{\left(-x_i^T \beta \right)}}
\end{gathered}
\end{equation}

In [2]:
# Simulating data
seed = 640985
torch.manual_seed(seed)
# a, b = 2, 10
n = 200
K = 4
# alpha = td.Gamma(a,b).sample()
sig = 2
betas = td.Normal(0, sig).sample([K,1])
X = torch.randn(n, K-1)
X = torch.cat((torch.ones(n,1), X), dim = 1)
p = torch.sigmoid(X@betas)
y = td.Bernoulli(p).sample()


In [3]:
import pickle
file = open("obs_blogisitcs","wb")
pickle.dump(y.numpy(), file)
file.close()

file = open("covariates_blogisitcs","wb")
pickle.dump(X.numpy(), file)
file.close()

In [3]:
def tensor_compliment(tensors, ind, dim = 0):
    len_ = tensors.size()[dim]
    mask = torch.ones(len_, dtype =torch.bool)
    mask[ind] = False
    if dim == 0:
        return tensors[ind,:], tensors[mask,:]
    elif dim == 1:
        return tensors[:,ind], tensors[:, mask]
    else:
        print("error with dimension")


In [4]:
def _test_logl(X, betas, y, ind):
    beta_, beta_c = tensor_compliment(betas, ind)
    X_, X_c = tensor_compliment(X, ind, dim = 1)
    p1, p2 = X_ @ beta_, X_c @ beta_c
    logits_ = (p1.unsqueeze(2) + p2.unsqueeze(1))
    bernoulli_ = td.Bernoulli(logits=logits_)
    logl = bernoulli_.log_prob(y.unsqueeze(2)) 

    return logl.sum(dim = 0)

def _test_logprior(betas, sig, ind):
    f = -0.5 * betas**2 / sig**2
    f_, f_c = tensor_compliment(f, ind)
    # prior_dens = td.Normal(loc = 0, scale = sig)
    # p1, p2 = prior_dens.log_prob(prior_dens), 
    return f_.sum(dim = 0).unsqueeze(1) + f_c.sum(dim = 0).unsqueeze(0)

def _test_logp(X, betas, sig, y, ind):
    logl = _test_logl(X, betas, y, ind)
    logprior = _test_logprior(betas, sig, ind)
    return logl + logprior

In [5]:
class MFVB_LMC_BayesLogistic(object):

    def __init__(self, X, y):
        self.X = X
        self.y = y
        self.n, self.K = X.size()
        self.split = int(X.size()[1]/2)
    
    def _logL(self, betas):
        """
        Compting the loglikelihood of the logistic model
        ------------------------------------------------
        # betas: the coefficients the first dimension is of intercept (K, N) 
        ------------------------------------------------
        output: the logL 
        """
        logits_ = self.X @ betas
        Bernoulli_ = td.Bernoulli(logits = logits_)
        return Bernoulli_.log_prob(self.y).sum(dim =0)
    
    def _logprior(self, betas, sig):
        """
        Computing the log-prior density of the simple bayes logisitc
        --------------------------------------------------
        # betas: the observed values of betas
        # sig: the common standard deviation for all betas under priors
        --------------------------------------------------
        """
        f = -0.5 * betas**2 / sig**2
        return f.sum(dim = 0)
    
    def _logposterior(self, betas, sig):
        """
        Computing the log-posterior density up to normalisng constant
        ------------
        # betas: the observed values of betas (K,s)
        # sig: the common standard deviation for all betas under priors (1,1)
        ------------
        output: the log-posterior
        """
        return self._logL(betas) + self._logprior(betas,sig)
    
    def _test_logl(self, betas, ind):
        beta_, beta_c = tensor_compliment(betas, ind)
        X_, X_c = tensor_compliment(self.X, ind, dim = 1)
        p1, p2 = X_ @ beta_, X_c @ beta_c
        logits_ = (p1.unsqueeze(2) + p2.unsqueeze(1))
        bernoulli_ = td.Bernoulli(logits=logits_)
        logl = bernoulli_.log_prob(self.y.unsqueeze(2)) 

        return logl.sum(dim = 0)

    def _test_logprior(self, betas, sig, ind):
        f = -0.5 * betas**2 / sig**2
        f_, f_c = tensor_compliment(f, ind)
        return f_.sum(dim = 0).unsqueeze(1) + f_c.sum(dim = 0).unsqueeze(0)

    def _test_logp(self, betas, sig, ind):
        logl = self._test_logl(betas, ind)
        logprior = self._test_logprior(betas, sig, ind)
        return logl + logprior

    def _test_mfvb_lmc_random(self, sig, iter = 150, N = 200, lr = 1e-2):
        ## Initialse point cloud
        beta_cloud = torch.randn(self.K,N, requires_grad= True)
        logp_trace = torch.zeros(iter)
        pbar = trange(iter)
        ind_ = torch.multinomial(torch.ones(self.K),self.split)
        logp = self._test_logp(beta_cloud, sig, ind_).mean(dim = 1)
        logp.sum().backward()
        for t in pbar:
            step = lr*100/(100 + 2*t)
            beta_cloud.grad.zero_()
            ind_ = torch.multinomial(torch.ones(self.K),self.split)
            logp = self._test_logp(beta_cloud, sig, ind_).mean(dim =1)
            logp.sum().backward()
            self._step2(beta_cloud, ind_, step)
            logp_trace[t] = logp.mean().item()
            pbar.set_postfix(logposterior = logp_trace[t].item())
        return beta_cloud, logp_trace
    
    def _barycenters_logl(self, betas, ind):
        beta_, beta_c = tensor_compliment(betas, ind)
        beta_c = beta_c.mean(dim = 1, keepdim = True)
        X_, X_c = tensor_compliment(self.X, ind, dim = 1)
        p1, p2 = X_ @ beta_, X_c @ beta_c
        logits_ = p1 + p2
        bernoulli_ = td.Bernoulli(logits=logits_)
        logl = bernoulli_.log_prob(self.y) 
        return logl.sum(dim = 0)

    def _barycenters_logprior(self, betas, sig, ind):
        beta_, beta_c = tensor_compliment(betas, ind)
        beta_c = beta_c.mean(dim = 1, keepdim = True)
        f_ = -0.5 * beta_**2 / sig**2
        f_c = -0.5 * beta_c**2 / sig**2
        return (f_ + f_c).sum(dim = 0)
    
    def _barycenters_logp(self, betas, sig, ind):
        logl = self._barycenters_logl(betas, ind)
        logprior = self._barycenters_logprior(betas, sig, ind)
        return logl + logprior
    
    def _barycenters_mfvb_lmc_iterated(self,sig, iter=150, N=200, lr = 1e-2):
        ## Initialse point cloud
        beta_cloud = torch.randn(self.K,N, requires_grad= True)
        logp_trace = torch.zeros(iter)
        pbar = trange(iter)
        logp = self._logposterior(beta_cloud, sig)
        logp.sum().backward()
        for t in pbar:
            step = lr*100/(100 + 2*t)
            for it in range(2):
                beta_cloud.grad.zero_()
                if it == 0:
                    index_ = torch.arange(self.K)[:self.split]
                else:
                    index_ = torch.arange(self.K)[self.split:]
                logp = self._barycenters_logp(beta_cloud, sig, index_)
                logp.sum().backward()
                
                self._step2(beta_cloud,index_, step)
            logp_trace[t] = logp.mean().item()
            pbar.set_postfix(logposterior = logp_trace[t].item())
        return beta_cloud, logp_trace

    def _test_mfvb_lmc_iterated(self,sig, iter=150, N=200, lr = 1e-2):
        ## Initialse point cloud
        beta_cloud = torch.randn(self.K,N, requires_grad= True)
        logp_trace = torch.zeros(iter)
        pbar = trange(iter)
        logp = self._logposterior(beta_cloud, sig)
        logp.sum().backward()
        for t in pbar:
            step = lr*100/(100 + 2*t)
            for it in range(2):
                beta_cloud.grad.zero_()
                if it == 0:
                    index_ = torch.arange(self.K)[:self.split]
                else:
                    index_ = torch.arange(self.K)[self.split:]
                logp = self._test_logp(beta_cloud, sig, index_).mean(dim = 1)
                logp.sum().backward()
                
                self._step2(beta_cloud,index_, step)
            logp_trace[t] = logp.mean().item()
            pbar.set_postfix(logposterior = logp_trace[t].item())
        return beta_cloud, logp_trace

    def _step(self,betas, lr = 1e-2):
        weights = torch.ones(1,self.K)
        index = torch.multinomial(weights, self.split).squeeze()
        noise_ = torch.randn(self.split, betas.size()[1])
        with torch.no_grad():
            betas[index,:] = betas[index,:].add_(lr* betas.grad[index,:] + (2*lr)**0.5 * noise_)
        
    def _mfvb_lmc_random(self, sig, iter = 150, N = 200, lr = 1e-2):
        ## Initialse point cloud
        beta_cloud = torch.randn(self.K,N, requires_grad= True)
        logp_trace = torch.zeros(iter)
        pbar = trange(iter)
        logp = self._logposterior(beta_cloud, sig)
        logp.sum().backward()
        for t in pbar:
            step = lr*100/(100 + 2*t)
            beta_cloud.grad.zero_()
            logp = self._logposterior(beta_cloud, sig)
            logp.sum().backward()
            self._step(beta_cloud, step)
            logp_trace[t] = logp.mean().item()
            pbar.set_postfix(logposterior = logp_trace[t].item())
        return beta_cloud, logp_trace
    
    def _step2(self, betas, index, lr= 1e-2):
        """
        Step without randomly choosing dimensions to step
        """
        noise_ = torch.randn(betas.size())
        with torch.no_grad():
            betas[index,:] = betas[index,:] + lr*betas.grad[index,:] + (2*lr)**0.5 * noise_[index,:]

    def _mfvb_lmc_iterated(self,sig, iter=150, N=200, lr = 1e-2):
        ## Initialse point cloud
        beta_cloud = torch.randn(self.K,N, requires_grad= True)
        logp_trace = torch.zeros(iter)
        pbar = trange(iter)
        logp = self._logposterior(beta_cloud, sig)
        logp.sum().backward()
        for t in pbar:
            step = lr*100/(100 + 2*t)
            for it in range(2):
                beta_cloud.grad.zero_()
                logp = self._logposterior(beta_cloud, sig)
                logp.sum().backward()
                if it == 0:
                    index_ = torch.arange(self.K)[:self.split]
                else:
                    index_ = torch.arange(self.K)[self.split:]
                self._step2(beta_cloud,index_, step)
            logp_trace[t] = logp.mean().item()
            pbar.set_postfix(logposterior = logp_trace[t].item())
        return beta_cloud, logp_trace



In [6]:
rc_ = MFVB_LMC_BayesLogistic(X,y) 
betas_bc, trace_bc = rc_._barycenters_mfvb_lmc_iterated(2, iter = 300, N = 3000)

  0%|          | 0/300 [00:00<?, ?it/s]

In [ ]:
Ns = [200, 500, 1000, 2000, 5000]
vars_ = torch.zeros(5, 4)
for i in range(len(Ns)):
    betas_pointcloud, _ = rc_._mfvb_lmc_iterated(2, iter =300, N =Ns[i])
    vars_[i,:] = betas_pointcloud.var(dim = 1)

In [107]:
import pickle
mcmc_trace = pickle.load(open("pymc_bayes_logisitic", "rb"))
# fig = px.histogram(mcmc_trace[0,:], histnorm="probability")
# fig.show()
betas_ = betas_bc

In [108]:
import plotly.figure_factory as ff
import numpy as np

x1 = mcmc_trace[0,:]
x2 = betas_[0,:].detach().numpy()
print(f"variance of MCMC particles: {x1.var()} \n varianace of VB: {x2.var()}")

hist_data = [x1, x2]

group_labels = ['MCMC', 'VB']
colors = ['#333F44', '#37AA9C']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data
                        , group_labels
                        , show_hist=False
                        , colors=colors)

# Add title
fig.update_layout(title_text= "Comparison MFVB-LMC vs HMC: Intercept"
                , width = w + 150, height = h + 100
                , margin = margin_dic)
fig.show()
fig.write_image("images/beta0_bayeslogistics.pdf")

variance of MCMC particles: 0.14171807501196795 
 varianace of VB: 0.11162441696903767


In [109]:
x1 = mcmc_trace[1,:]
x2 = betas_[1,:].detach().numpy()
print(f"variance of MCMC particles: {x1.var()} \n varianace of VB: {x2.var()}")
hist_data = [x1, x2]

group_labels = ['MCMC', 'VB']
colors = ['#333F44', '#37AA9C']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title

fig.update_layout(title_text= "Comparison MFVB-LMC vs HMC: Beta1"
                , width = w+ 150, height = h + 100
                , margin = margin_dic)
fig.show()
fig.write_image("images/beta1_bayeslogistics.pdf")

variance of MCMC particles: 0.3381157315012018 
 varianace of VB: 0.22858226340603915


In [110]:
x1 = mcmc_trace[2,:]
x2 = betas_[2,:].detach().numpy()
print(f"variance of MCMC particles: {x1.var()} \nvarianace of VB: {x2.var()}")
hist_data = [x1, x2]

group_labels = ['MCMC', 'VB']
colors = ['#333F44', '#37AA9C']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title
fig.update_layout(title_text= "Comparison MFVB-LMC vs HMC: Beta2"
                , width = w+ 150, height = h + 100
                , margin = margin_dic)
fig.show()
fig.write_image("images/beta2_bayeslogistics.pdf")

variance of MCMC particles: 0.0752215854803227 
varianace of VB: 0.07209355408891965


In [111]:
x1 = mcmc_trace[3,:]
x2 = betas_[3,:].detach().numpy()
print(f"variance of MCMC particles: {x1.var()} \nvarianace of VB: {x2.var()}")

hist_data = [x1, x2]

group_labels = ['MCMC', 'VB']
colors = ['#333F44', '#37AA9C']

# Create distplot with curve_type set to 'normal'
fig = ff.create_distplot(hist_data, group_labels, show_hist=False, colors=colors)

# Add title
fig.update_layout(title_text= "Comparison MFVB-LMC vs HMC: Beta3"
                , width = w+ 150, height = h + 100
                , margin = margin_dic)
fig.show()
fig.write_image("images/beta3_bayeslogistics.pdf")

variance of MCMC particles: 0.11146466803733587 
varianace of VB: 0.07936384756001987
